# Encryption: AES and RSA

**Objective:** Create a Python script to encrypt and decrypt text data using symmetric (AES) and asymmetric (RSA) encryption techniques.

Run the single code cell in Google Colab.

In [ ]:
!pip install pycryptodome -q
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import base64

def pad(b):
    pad_len = AES.block_size - (len(b) % AES.block_size)
    return b + bytes([pad_len]) * pad_len

def unpad(b):
    return b[:-b[-1]]

def aes_encrypt(key, plaintext):
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct = cipher.encrypt(pad(plaintext.encode()))
    return base64.b64encode(iv + ct).decode()

def aes_decrypt(key, b64cipher):
    raw = base64.b64decode(b64cipher)
    iv = raw[:16]
    ct = raw[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct))
    return pt.decode()

def rsa_generate(bits=2048):
    key = RSA.generate(bits)
    private_pem = key.export_key()
    public_pem = key.publickey().export_key()
    return private_pem, public_pem

def rsa_encrypt(public_pem, plaintext):
    key = RSA.import_key(public_pem)
    cipher = PKCS1_OAEP.new(key)
    ct = cipher.encrypt(plaintext.encode())
    return base64.b64encode(ct).decode()

def rsa_decrypt(private_pem, b64cipher):
    key = RSA.import_key(private_pem)
    cipher = PKCS1_OAEP.new(key)
    ct = base64.b64decode(b64cipher)
    pt = cipher.decrypt(ct)
    return pt.decode()

aes_key = get_random_bytes(16)
sample_text = "Hello, this is a secret message."

aes_cipher = aes_encrypt(aes_key, sample_text)
aes_plain = aes_decrypt(aes_key, aes_cipher)

priv, pub = rsa_generate()
rsa_cipher = rsa_encrypt(pub, sample_text)
rsa_plain = rsa_decrypt(priv, rsa_cipher)

print("AES key (base64):", base64.b64encode(aes_key).decode())
print("AES encrypted:", aes_cipher)
print("AES decrypted:", aes_plain)
print()
print("RSA public key (PEM):")
print(pub.decode()[:200], "...")
print("RSA encrypted:", rsa_cipher)
print("RSA decrypted:", rsa_plain)
